# Nonlinear analysis -Assignment 2

04.10.2023 - Jérémie Engler and Laure Toullier

## Part b

### Imports

In [1]:
import numpy as np

Definitions

In [16]:
# Element connectivity
nelements = 2 
nnodes = 3

# Written input:
connectivity = np.array([[1, 2], [2, 3]])
nodes_P = np.array([2])
rest=[1,1,0,0,1,1] #restraints
DOF=np.size(rest)

# Unit stiffness matrix of bar in local coordinate system
K_unit = np.array([[1, 0, -1, 0], 
                   [0, 0,  0, 0], 
                   [-1, 0, 1, 0], 
                   [0, 0, 0, 0]])

# Input data 
A = [15e3,2e3] #mm^2 Cross-sectional area of the elements
a = 3000 #mm Half lenght of the truss
b = 4000 #mm Height of the truss
P = 8625e3 #N Load
phiP = 60 #° Angle of load
theta = [53.13, -53.13] #° Angle of elements, at first node of the element
Pglob = np.array([P*np.cos(np.radians(phiP)), P*np.sin(np.radians(phiP))]) #N P force in global coordinates (x,y)

rest=[1,1,0,0,1,1]

#Material stiffness
E = [200e3, 200e3]  # N/mm^2 Young modulus of the elements
fy= [500e6, 500e6] #Pa
ey=np.array([fy[0]/E[0], fy[1]/E[1]])
alpha=0.02


Stiffness

In [17]:
T = np.zeros((nelements,4,4))
L = []
k = []
k_glob = np.zeros((nelements,4,4))

for i in range(nelements):
    c = np.cos(np.radians(theta[i]))
    s = np.sin(np.radians(theta[i]))
    Li = abs(a/c)
    if theta[i] == -90:
        Li = b
    Ai = A[i]
    Ei = E[i]
    ki = Ei * Ai / Li
    T[i] = np.array([[c, s, 0, 0], 
              [-s, c, 0, 0], 
              [0, 0, c, s], 
              [0, 0, -s, c]])
    L.append(Li)
    k.append(ki)

for i in range(nelements):
    k_glob[i] = k[i] * T[i].T @ K_unit @ T[i]


# Assemble global stiffness matrix
NDoF = 2*nnodes
K_global = np.zeros((NDoF,NDoF))
K_globals = np.zeros((nelements,NDoF, NDoF))


# Assemble elements
for i in range (nelements):
    connectelem = connectivity[i] #nodes connected to the element
    indexs = 2*connectelem
    i11 = indexs[0]-2
    i12 = indexs[0]
    i21 = indexs[1]-2
    i22 = indexs[1]

    Ki_global = np.zeros((NDoF, NDoF))
    ki_global = k_glob[i]
    Ki_global[i11:i12, i11:i12] = ki_global[0:2, 0:2]
    Ki_global[i21:i22, i11:i12] = ki_global[2:4, 0:2]
    Ki_global[i11:i12, i21:i22] = ki_global[0:2, 2:4]
    Ki_global[i21:i22, i21:i22] = ki_global[2:4, 2:4]
    K_global = K_global + Ki_global
    K_globals[i] = Ki_global

System equilibrium

In [18]:
# Calculate displacements at nodes with forces applied

def displacements (nodes_P, P_glob, K_global):
    numnodesP = len(nodes_P)
    F_red = Pglob
    indexssup = 2*nodes_P - 1
    indexsinf = indexssup - 1
    mask = []
    for i in range(numnodesP):
        mask.append(indexsinf[i])
        mask.append(indexssup[i])
    K_red = K_global[mask,:]
    K_red = K_red[:,mask]

    u_red = np.linalg.inv(K_red) @ F_red
    return(u_red)


# Calculate reaction forces at other nodes

def reaction_forces(nnodes, numnodesP, u_red,K_global):
    u_vec = np.zeros((2*nnodes))

    for i in range(numnodesP):
        u_vec[indexsinf[i]] = u_red[2*i]
        u_vec[indexssup[i]] = u_red[2*i+1]

    F = K_global @ u_vec
    return (F)


# Calculate axial forces
def axial_force(nelelements, K_global,u_vec, connectivity):
    axialF = []
    for i in range(nelements):
        F_elei = K_globals[i] @ u_vec
        index_red = 2*connectivity[i]
        F_elei_red = F_elei[[index_red[0]-2, index_red[0]-1, index_red[1]-2, index_red[1]-1]]
        F_elei_local = T[i] @ F_elei_red
        Fi = F_elei_local[2]
        axialF.append(Fi)
    return (axialF)

for i in range(nelements):
    print(f"Axial load in element {i+1}: {round(axialF[i])} N")
    print()

for i in range(nnodes):
    if i == 0:
        print("Left support:")
    if i == 1:
        print("Right support:")
    print(f"Node {i+1}, horizontal displacement: {round(u_vec[2*i],2)} mm")
    print(f"Node {i+1}, vertical displacement: {round(u_vec[2*i+1],2)} mm")
    print()
    print(f"Node {i+1}, horizontal force: {round(F[2*i])} N")
    print(f"Node {i+1}, vertical force: {round(F[2*i+1])} N")
    print()
    print()

Axial load in element 1: 8262166 N

Axial load in element 2: 1074683 N

Left support:
Node 1, horizontal displacement: 0.0 mm
Node 1, vertical displacement: 0.0 mm

Node 1, horizontal force: -4957311 N
Node 1, vertical force: -6609724 N


Right support:
Node 2, horizontal displacement: 0.28 mm
Node 2, vertical displacement: 17.0 mm

Node 2, horizontal force: 4312500 N
Node 2, vertical force: 7469469 N


Node 3, horizontal displacement: 0.0 mm
Node 3, vertical displacement: 0.0 mm

Node 3, horizontal force: 644811 N
Node 3, vertical force: -859745 N




Newton Raphson Method

In [19]:
#Initialisation

u_0=np.array([0,0,0,0,0,0]) #initial nodal displacement
K_0=K_global #initial stiffness matrix
load_incr=2 #increment for loadstep
np.shape(K_0)

(6, 6)

In [21]:
#iteration 1:

P_external=Pglob/load_incr
delta_u=np.linalg.inv(K_0)@P_external
u_1=u_0+delta_u

Pr=axial_load(nelelements, K_global,u_1, connectivity)

R=P_external-Pr

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 6)